In [28]:
#Install libraries
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import json

In [29]:
# Upload credentials
with open('credentials.json', 'r') as file:
    credentials = json.load(file)


master_username = credentials['master_username']
master_user_password = credentials['master_user_password']
data_base_name = credentials['data_base_name']
region_name = credentials['region_name']

In [30]:
def create_s3_bucket(bucket_name, region_name=region_name):
    """
    Create an S3 bucket
    
    :param bucket_name: unique name for bucket
    :param region_name: region for bucket
    :return: True if the bucket was created successfully, False otherwise
    """
    # Create client S3
    s3_client = boto3.client('s3', region_name=region_name)

    try:
        if region_name == "us-east-1":
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region_name}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)

        print(f"Bucket '{bucket_name}' successfully created in the region {region_name}.")
        return True
    except ClientError as e:
        print(f"Error creating bucket: {e}")
        return False

In [31]:
def upload_files_to_s3(file_paths, bucket_name):
    """
    Upload files to S3 bucket, creating a folder for each file based on its root name.

    :param file_paths: path of local files.
    :param bucket_name: S3 bucket name.
    :return: None
    """
    # Create client S3
    s3_client = boto3.client('s3')

    for local_file_path in file_paths:
        try:
            # Get file name without extension to use as folder in S3
            file_name = os.path.basename(local_file_path)
            folder_name = os.path.splitext(file_name)[0]
            s3_file_key = f"{folder_name}/{file_name}"

            # Upload files
            s3_client.upload_file(local_file_path, bucket_name, s3_file_key)
            print(f"File {local_file_path} upload to {bucket_name}/{s3_file_key} successfully.")
        
        except FileNotFoundError:
            print(f"File {local_file_path} was not found.")
        except NoCredentialsError:
            print("Credentials are not available.")
        except PartialCredentialsError:
            print("Credentials are incomplete.")
        except Exception as e:
            print(f"Error uploading file {local_file_path}: {e}")

In [32]:
def create_rds_instance(db_instance_identifier, master_username, master_user_password, db_name, region_name):
    # Create client RDS
    rds_client = boto3.client('rds', region_name=region_name)
    
    try:
        # Create instance RDS
        response = rds_client.create_db_instance(
            DBInstanceIdentifier=db_instance_identifier,
            DBInstanceClass='db.t4g.micro',
            Engine='mysql',
            MasterUsername=master_username,
            MasterUserPassword=master_user_password,
            AllocatedStorage=20,
            PubliclyAccessible=False,
            MultiAZ=False,
            StorageType='gp2',
            BackupRetentionPeriod=7,
            DBName=db_name,
        )
        print("Instance creation started:", response)
    except Exception as e:
        print("Error creating RDS instance:", e)

In [ ]:
# Create the bucket
bucket_name = "big-data-migration-globant"

create_s3_bucket(bucket_name, region_name)

Bucket 'big-data-migration-globant' successfully created in the region us-east-1.


True

In [34]:
#Upload the files to bucket
file_paths = [
    "departments.csv",
    "hired_employees.csv",
    "jobs.csv"
]

upload_files_to_s3(file_paths, bucket_name)

File departments.csv upload to big-data-migration-globant/departments/departments.csv successfully.
File hired_employees.csv upload to big-data-migration-globant/hired_employees/hired_employees.csv successfully.
File jobs.csv upload to big-data-migration-globant/jobs/jobs.csv successfully.


In [ ]:
#Create the RDS, mysql
create_rds_instance(
    db_instance_identifier="db-globant",
    master_username=master_username,
    master_user_password=master_user_password,
    db_name=data_base_name,
    region_name=region_name
)

Instance creation started: {'DBInstance': {'DBInstanceIdentifier': 'big-data-migration-instance', 'DBInstanceClass': 'db.t4g.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'creating', 'MasterUsername': 'jennifer_castaneda', 'DBName': 'db_nombre_unico', 'AllocatedStorage': 20, 'PreferredBackupWindow': '06:07-06:37', 'BackupRetentionPeriod': 7, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0a65238c88f1dd91d', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-06bb97ddaf1702626', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-0b3f7a394fdffdef1', 'SubnetAvailabilityZone': {'Name': 'us-east-1b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0949e25470fa007c1', 'SubnetAvailabilityZone': {'Name': 'us-east-1d'}, 'SubnetOutpost': {}, '